In [2]:
print ( 'hello world')

hello world


In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/ad_feature.csv',header=0)

In [4]:
df.head()

,adgroup_id,cate_id,campaign_id,customer,brand,price
0,63133,6406,83237,1,95471.0,170.00
1,313401,6406,83237,1,87331.0,199.00
2,248909,392,83237,1,32233.0,38.00
3,208458,392,83237,1,174374.0,139.00
4,110847,7211,135256,2,145952.0,32.99


In [5]:
np.linspace(0,1,11)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [6]:
c,  bins = pd.qcut( df['price'], q = np.linspace(0,1,11) ,retbins=True,  duplicates='drop')  
price_level, ind  = pd.factorize(c, sort=True)
df['price_level'] = price_level 


In [7]:
df.head()

,adgroup_id,cate_id,campaign_id,customer,brand,price,price_level
0,63133,6406,83237,1,95471.0,170.00,5
1,313401,6406,83237,1,87331.0,199.00,6
2,248909,392,83237,1,32233.0,38.00,1
3,208458,392,83237,1,174374.0,139.00,4
4,110847,7211,135256,2,145952.0,32.99,1


In [9]:
## 离群点处理 
des = df['price'].describe() 
print(des)


count    8.468110e+05
mean     1.838867e+03
std      3.108877e+05
min      1.000000e-02
25%      4.900000e+01
50%      1.390000e+02
75%      3.520000e+02
max      1.000000e+08
Name: price, dtype: float64


In [10]:
valid_max =  des['50%'] + 3* ( des['75%'] -des['50%'] ) 

df['price'] = df['price'].clip(0, valid_max)   

#观察下输出的最大值，已经把异常值给clip了
print( df['price'].describe())


count    846811.000000
mean        243.996387
std         253.800315
min           0.010000
25%          49.000000
50%         139.000000
75%         352.000000
max         778.000000
Name: price, dtype: float64


In [11]:

####### 接下来统计下 各个品牌的平均价格 
g = df.groupby('brand')
m = g['price'].mean() 

#只看下前10行
m[:10] 


brand
2.0      53.412500
3.0     739.368421
7.0     217.636364
8.0      32.450000
9.0      44.200000
10.0    158.916667
Name: price, dtype: float64

In [12]:
## 这里用另外一个简单数据来介绍下 axis 概念，很多复杂操作依赖于对这个的理解  
a = { 'A' : [ 1, 2, 3 ] , 'B' : [100,200, 300 ]} 
d = pd.DataFrame( a ) 
print(d)


   A    B
0  1  100
1  2  200
2  3  300


In [13]:
d.sum(axis=0 )  #axis = 0表示 按照行遍历进行sum  

A      6
B    600
dtype: int64

In [14]:
d.sum(axis=1)  # axis=1 表示 按照列遍历 进行sum  

0    101
1    202
2    303
dtype: int64

In [15]:
# 再来看一下d ，参照一下
d

,A,B
0,1,100
1,2,200
2,3,300


In [16]:
d['sum'] =  d.sum(axis=1)
d

,A,B,sum
0,1,100,101
1,2,200,202
2,3,300,303


In [17]:
f =  d.sum(axis=0)  
f

A        6
B      600
sum    606
dtype: int64

In [23]:
d=  d.append(f,ignore_index=True)
print(d)

   A    B  sum
0  1  100  101
1  2  200  202
2  3  300  303
3  6  600  606


In [24]:
d.index = [ 0 , 1, 2 , 'sum'] 
print(d)

     A    B  sum
0    1  100  101
1    2  200  202
2    3  300  303
sum  6  600  606
